# Naive Bayes en Scikit Learn

En este _notebook_ vamos a aprender a trabajar con el modelo **Multinomial Naive Bayes** en Scikit Learn. Lo primero que haremos será importar el _dataset_ con el que vamos a trabajar. Corresponde a un _dataset_ de correos, en el que hay correos deseados (fáciles de detectar), correos deseados (difíciles de detectar) y correos no deseados (_spam_). Para cargar los correos pasamos el _path_ de la carpeta `mails-raw` como se ve más abajo, que es la carpeta que encontrarás junto a este _notebook_ y que contiene los correos.

In [1]:
import glob

PATH_MAILS = 'mails-raw/*/*'

mails = []
answers = []

for filename in glob.glob(PATH_MAILS):
    # Checkeamos si es spam o no
    is_spam = 'ham' not in filename
    
    # Ignoramos los posibles errores al abrir un archivo
    with open(filename, errors='ignore') as mail_file:
        for line in mail_file:
            if line.startswith("Subject:"):
                # Hacemos strip desde la izquierda
                subject = line.lstrip("Subject: ")
                mails.append(subject.strip())
                if is_spam:
                    answers.append(1)
                else:
                    answers.append(0)
                break


Vamos a explorar algunos correos. Notemos que estamos trabajando solamente con el asuunto del correo.

In [2]:
mails[0:10]

['Friend, Copy ANY DVD or Playstation Game with this software......',
 '5% Guaranteed for Eight Years',
 'Congratulations! You Get a Free Handheld Organizer!',
 'One of a kind Money maker! Try it for free!',
 'Online Doctors will fill your Viagra Prescription Now!!!                QEEB',
 'Take your Marketing to the Next Level',
 'One Sale - Three Commission Streams',
 'Find Peace, Harmony, Tranquility, And Happiness Right Now!',
 'ADV: Extended Auto Warranties Here                                                    undoc',
 'Definitely the answer many have been waiting for!!']

Y vamos a ver la respuesta a estas instancias.

In [4]:
answers[0:10]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

Si bien en clases vimos cómo construir un clasificador _Naive Bayes_, en la práctica este funciona un poco distinto. Lo más importante es que en la fase de entrenamiento este clasificador espera recibir un _dataset_ pre-procesado, que es una matriz que tiene un correo en cada fila y en la columna tiene todo nuestro vocabulario observado de palabras. En la posición $(i,j)$ (fila, columna) tenemos el número de veces que la palabra $j$ aparece en el correo $i$. Este procesamiento lo podemos hacer rápidamente con el objeto `CountVectorizer`.

Primero, vamos a unir el _dataset_ y separarlo en una parte de **entrenamiento** y otra de **prueba**. Luego armamos el vocabulario con la clase `CountVectorizer` sobre el _dataset_ de entrenamiento. **Ojo**: es importante que al momento de apicar esta transformación al _dataset_, solo lo hagamos con los datos de entrenamiento, ya que si incluimos los datos de prueba, podemos estar filtrando información.

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

# Recordemos que la función train_test_split produce un sampleo
# representativo de las clases
X_train, X_test, y_train, y_test = train_test_split(mails, answers)

# Vamos a entrenar un CountVectorizer
vect = CountVectorizer()
vect.fit(X_train)

CountVectorizer()

Como vemos, solo preparamos el objeto de tipo `CountVectorizer`. Ahora, lo que vamos a hacer, es transformar los _dataset_ de entrenamiento y prueba.

In [5]:
X_train_vect = vect.transform(X_train)
# Notar que estamos transformando con el fit del dataset de entrenamiento
# No debemos re-entrenar con los datos de test
# Si una palabra no es conocida por el CountVectorizer simplemente será ignorada
X_test_vect = vect.transform(X_test)

Ahora vamos a explorar brevemente el _dataset_ de entrenamiento.

In [8]:
X_train_vect

<2475x3763 sparse matrix of type '<class 'numpy.int64'>'
	with 14316 stored elements in Compressed Sparse Row format>

Notamos que es de tipo _sparse matrix_, que es una forma de representar matrices poco densas. Vamos a ver la dimensión de esta matriz.

In [9]:
X_train_vect.shape

(2475, 3763)

Esto quiere decir que tenemos 2475 correos, y nuestro vocabulario está formado por 3763 palabras. Vamos a intentar encontrar cuáles son estas palabras.

In [10]:
vect.get_feature_names()

['00',
 '000',
 '0001015',
 '01',
 '02',
 '03',
 '04',
 '05',
 '05775748',
 '057sxua1524uhkc5',
 '06',
 '07',
 '08',
 '09',
 '0rvn',
 '10',
 '100',
 '1006',
 '1008',
 '100k',
 '1012',
 '1030',
 '1046',
 '1049',
 '1052',
 '1053',
 '1054',
 '1075',
 '1090',
 '10th',
 '10x',
 '11',
 '11058',
 '111bagy1254ihzj0',
 '11956',
 '12',
 '1200bps',
 '12021',
 '12144',
 '12173',
 '12304',
 '13',
 '131',
 '133',
 '136',
 '139',
 '14',
 '141',
 '148',
 '15',
 '150',
 '153',
 '16889',
 '17',
 '17441',
 '17639',
 '179dml',
 '18',
 '180',
 '18070',
 '19',
 '1987',
 '199',
 '1rh7',
 '20',
 '200',
 '2000',
 '2002',
 '2003',
 '2004',
 '2009vjtt6',
 '200gb',
 '2022',
 '20gb',
 '21',
 '21198',
 '21st',
 '22',
 '22311',
 '225',
 '226',
 '227',
 '229',
 '23',
 '23875',
 '24',
 '24344',
 '2453',
 '24772',
 '25',
 '250',
 '256mb',
 '26',
 '261uccu3485ggcz0',
 '26792',
 '27',
 '28',
 '2800',
 '28940',
 '29',
 '30',
 '300',
 '309',
 '30k',
 '31',
 '3154',
 '32',
 '32004',
 '32053',
 '32482',
 '33',
 '34',
 '349',

Estas son las columnas en orden. Y ahora si pedimos un correo en particular.

In [38]:
import numpy as np

X_train_vect[0].todense()

matrix([[0, 0, 0, ..., 0, 0, 0]])

Como vemos, practicamente solo tenemos 0s. Vamos a ver las posicione donde hay valores no 0.

In [54]:
import pandas as pd

df = pd.DataFrame(X_train_vect[0].todense().T, columns=['x'])
df[df['x'] != 0]

,x
580,1
1229,1
2743,1


Esto quiere decir que primer correo contenía solo las palabras 580, 1229 y 2743. Estas serían las siguientes.

In [43]:
vect.get_feature_names()[580]

'bug'

In [44]:
vect.get_feature_names()[1229]

'exmh'

In [45]:
vect.get_feature_names()[2743]

're'

Ahora instanciamos nuestro modelo, que corresponde a un _Multinomial Naive Bayes_.

In [47]:
from sklearn.naive_bayes import MultinomialNB

nb = MultinomialNB()

Y lo entrenamos.

In [48]:
nb.fit(X_train_vect, y_train)

MultinomialNB()

Vamos a predecir sobre todo el _dataset_ de prueba.

In [49]:
y_pred_class = nb.predict(X_test_vect)

Y ahora evaluamos su _accuracy_.

In [50]:
from sklearn import metrics

metrics.accuracy_score(y_test, y_pred_class)

0.9115151515151515

Y ahora mostramos la matriz de confusión, que es de la forma:

```
[ TN FP
  FN TP ]
```

In [51]:
metrics.confusion_matrix(y_test, y_pred_class)

array([[684,  39],
       [ 34,  68]])

Recordemos además que para ver la función de decisión lo podemos hacer con `predict_proba`, ya que aquí tenemos una probabilidaad, y no una función de decisión como habíamos visto hasta ahora.

In [53]:
nb.predict_proba(X_test_vect)

array([[9.99998298e-01, 1.70204086e-06],
       [9.99919311e-01, 8.06888371e-05],
       [9.99999131e-01, 8.68735424e-07],
       ...,
       [9.93179622e-01, 6.82037759e-03],
       [9.27750919e-01, 7.22490807e-02],
       [9.65845045e-01, 3.41549548e-02]])

Donde la probabilidad de ser spam es la segunda columna.

## Usando TF-IDF

Una forma de mejorar el desempeño es usar `TfidfVectorizer`. Esta transformación hace que las palabras que sean muy comunes (ej. artículos, pronombres, ...) ponderen menos en nuestra decisión. Investigar la idea de esta transformación será parte de lo que tendrás que hacer en tu tarea. Ahora veamos cómo transformar el _dataset_ con esta técnica.

In [55]:
from sklearn.feature_extraction.text import TfidfVectorizer

vect_tfidf = TfidfVectorizer()
vect_tfidf.fit(X_train)
X_train_tfidf = vect_tfidf.transform(X_train)
X_train_tfidf

<2475x3763 sparse matrix of type '<class 'numpy.float64'>'
	with 14316 stored elements in Compressed Sparse Row format>

Exploremos la fila que buscamos anteriormente.

In [56]:
df = pd.DataFrame(X_train_tfidf[0].todense().T, columns=['x'])
df[df['x'] != 0]

,x
580,0.654911
1229,0.714966
2743,0.244776


Notamos que los valores ya no son 1.